In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from dataclasses import *
from typing import *
import scipy.integrate as integrate
import unittest 
import timeit
import random
import sys
sys.path.insert(0, '/Users/niyi/Documents/GitHub/Optimal-Control/Tools')
from EnforceTyping import enforce_method_typing, EnforceClassTyping
from MDPFramework import MDPEnvironment, MDPController, LearningAgent

In [ ]:
@dataclass(kw_only=True)
class FiniteStateAutomaton(MDPEnvironment):
  n_states: int
  
  @dataclass(unsafe_hash=True, order=True)
  class State:
    """
    Represents a single node in the network.
    """
    id: Union[str, int, float, np.ndarray]
    neighbors: tuple

    def add_neighbor(self, neighbor: 'State'):
      """
      Adds a neighbor to the node's list of neighbors.
      """
      self.neighbors = self.neighbors+ (neighbor,)
      self.neighbors.append(neighbor)

    def remove_neighbor(self, neighbor: 'State'):
      """
      Removes a neighbor from the node's list of neighbors.
      """
      if neighbor in self.neighbors:
        self.neighbors.remove(neighbor)

  state_space: Set[State]= None
  initial_state: State= None
  current_state: State= None

  def __post_init__(self):
    self.state_space= self.randomly_connect_state_space(self.generate_states_space(self.n_states), 5)
    self.initial_state= random.choice(self.state_space)
    self.current_state= self.initial_state
  
  def add_node(self, node: State):
    """
    Adds a new node to the network.
    """
    self.state_space.add(node)

  def generate_states_space(self, n_states: int)-> Set[State]:
    state_space= set()
    for i in range(n_states):
      state_space.add(self.State(id= i, neighbors=()))
    return state_space

  def randomly_connect_state_space(self, state_space: Set[State], n_connections: int)-> Set[State]:
    for _ in range(n_connections):
      random_state1, random_state2= random.sample(sorted(state_space), 2)
      if self.are_connected(random_state1, random_state2):
          self.connect_nodes(random_state1, random_state2)
    return state_space

  def connect_nodes(self, state1: State, state2: State):
    """
    Connects two nodes in the network by adding each other to their respective neighbor lists.
    """
    state1.add_neighbor(state2)
    state2.add_neighbor(state1)

  def are_connected(self, state1: State, state2: State):
     are_connected= state1 not in state2.neighbors and state2 not in state1.neighbors
     return are_connected
  
  def disconnect_nodes(self, state1: State, state2: State):
    """
    Disconnects two nodes in the network by removing each other from their respective neighbor lists.
    """
    state1.remove_neighbor(state2)
    state2.remove_neighbor(state1)

  def find_node(self, id: Union[str, int, float, np.ndarray]):
    """
    Finds a node in the network by its name.
    """
    for node in self.nodes:
        if node.id == id:
            return node
    return None
  
  def transition_model(self, state: State, action: np.ndarray):
    ...

  def reward_model(self, state: State, action: np.ndarray, next_state: State, terminal_signal: bool)-> float:
    '''This is a scalar performance metric.'''
    ...

  def is_terminal_condition(self, state: State)-> bool:
    ...

  def transition_step(self, state: State, action: np.ndarray)-> tuple[State, float, bool]:
    ...

  def sample_trajectory(self, runtime: float)-> list[State]:
    ...


In [ ]:
test_node_network= FiniteStateAutomaton(n_states=10)


**1. Unit tests**

Unit tests are very low level and close to the source of an application. They consist in testing individual methods and functions of the classes, components, or modules used by your software. Unit tests are generally quite cheap to automate and can run very quickly by a continuous integration server.



**2. Integration tests**

Integration tests verify that different modules or services used by your application work well together. For example, it can be testing the interaction with the database or making sure that microservices work together as expected. These types of tests are more expensive to run as they require multiple parts of the application to be up and running.



In [ ]:
@dataclass
class RandomPolicy:
  action_dims: tuple
  control_magnitude: float

  def __call__(self, observation: np.ndarray):
    return self.control_magnitude* (2*np.random.ranf(self.action_dims)-1)
    
@dataclass
class RandomController(MDPController):
  control_magnitude: float
  
  @property
  def policy(self):
    return RandomPolicy(action_dims=self.environment.action_dims, control_magnitude=self.control_magnitude)
  
  @enforce_method_typing
  def act(self, observation: np.ndarray)-> np.ndarray:
      action= self.policy(observation)
      return action
  
  def observe(self, state)-> np.ndarray:
    observation= state.vector()
    return observation
  
  @enforce_method_typing
  def sample_trajectory(self, runtime: float, n_steps: int=100):
    trajectory= []
    trajectory_return= 0.0
    time= 0.0
    state= self.environment.initial_state
    time_interval= runtime/n_steps
    time_span = np.linspace(time, runtime, n_steps)
    for _ in time_span:
        observation= self.observe(state)
        trajectory.append(observation)
        action= self.act(observation)
        state, reward, _= self.environment.transition_step(state, action, time_interval)
        trajectory_return += reward
    return trajectory, trajectory_return
  
  def plot_trajectory(self, trajectory):
    trajectory= np.array(trajectory)
    px, py, vx, vy= trajectory.transpose()
    plt.figure(figsize=(8, 8))
    plt.plot(px, py, label='Trajectory')
    plt.scatter(px[0], py[0], c='k', marker='o', label='Start')
    plt.scatter(px[-1], py[-1], c='r', marker='*', label='End')
    xmax= max(abs(px))
    ymax= max(abs(py))
    true_max= max((xmax, ymax))
    plt.xlim(-2*true_max, 2*true_max)
    plt.ylim(-2*true_max, 2*true_max)
    plt.grid(True)
    plt.legend()
    plt.show()


**3. Functional tests**
Functional tests focus on the application requirements of the code. Functional tests are performed to check if this module functions as intended.They only verify the output of an action and do not check the intermediate states of the system when performing that action. 

**4. Performance testing**
Performance tests help to measure the reliability, speed, scalability, and responsiveness of an application. It can determine if an application meets performance requirements, locate bottlenecks, measure stability during peak traffic, and more.